# Bibliotecas e funções auxiliares

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os.path as osp
import pandas as pd
import re

from itertools import pairwise
from urllib.parse import urlparse

In [ ]:
DEFAULT_BINS = [0,1,2,5,10]
DEFAULT_N_TOP_SITES = 10

In [ ]:
def clean_models(models: list):
    stop_words = ['+',  '120hz', '2', '4g', '5g', 'amarelo', 'azul',
                  'basic', 'bluetooth', 'br', 'branco', 'cinza', 'de',
                  'earbuds', 'earphone', 'earphones', 'escuro', 'fone',
                  'gradiente', 'inteligente', 'laranja', 'mi', 'prata',
                  'preto', 'pulseira', 'rosa', 'roxo', 'tela', 'true',
                  'verde', 'wireless', 'wireless']

    pattern = '(([\d]+gb)?[\d]+gb)|(\([\d\w-]+\))|(\d+,\d+)|([+"\/\',’])'
    models = [model.lower() for model in models]
    models = [re.sub(pattern,'',model) for model in models]
    models = list(set([' '.join([token for token in model.split() if token not in stop_words]) for model in models]))
    
    return models

In [ ]:
def extract_site(url):
    if url:
        parsed_url = urlparse(url).netloc
        for prefix in ['www.', 'empresas.', 'm.', 'produto.', 'lista.', 'sp.']:
            if parsed_url.startswith(prefix):
                return parsed_url[len(prefix):]
        return parsed_url
    else:
        return '-'

In [ ]:
def plot_items_results(df):
    results = df[['original_query','results']].groupby('original_query').sum().values.reshape(-1,)
    item_count_no_results = np.where(results==0)[0].shape[0]
    item_count_results = np.where(results>0)[0].shape[0]
    total_results = results.sum()
    
    labels = [f'Itens com resultado:\n{item_count_results}', f'Itens sem resultado:\n{item_count_no_results}']
    values = [item_count_results, item_count_no_results]
    
    startangle = 0
    
    colors = [(0.17254901960784313, 0.6274509803921569, 0.17254901960784313, 0.85),
              (0.8392156862745098, 0.15294117647058825, 0.1568627450980392, 0.85)]
    
    fig, ax = plt.subplots(figsize=(5, 4), subplot_kw=dict(aspect="equal"))
    
    wedges, texts = ax.pie(values, wedgeprops=dict(width=0.5), startangle=startangle, colors=colors)
    
    bbox_props = dict(boxstyle="round,pad=0.3", fc="w", ec="k", lw=0.72)
    kw = dict(arrowprops=dict(arrowstyle="-"),
              bbox=bbox_props, zorder=0, va="center")
    
    kw = dict(arrowprops=dict(arrowstyle="-"),bbox=bbox_props,
              zorder=0, va="center")
    
    for i, p in enumerate(wedges):
        ang = (p.theta2 - p.theta1)/2. + p.theta1
        y = np.sin(np.deg2rad(ang))
        x = np.cos(np.deg2rad(ang))
        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = "angle,angleA=0,angleB={}".format(ang)
        kw["arrowprops"].update({"connectionstyle": connectionstyle})
        # horizontalalignment = 'center'
        ax.annotate(labels[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                    horizontalalignment=horizontalalignment, **kw)
    
    ax.annotate(f'{total_results:,d}\nresultados'.replace(',','.'), (0,0), horizontalalignment='center', verticalalignment='center')
    
    ax.set_title('Quantidade de itens pesquisados e resultados obtidos')
    plt.show()

In [ ]:
def plot_results_count(df,bins=DEFAULT_BINS):
    
    search_results_count = df_search_results_ean[['original_query','results']].groupby('original_query').sum().values.reshape(-1,)
    heights,_ = np.histogram(search_results_count,bins=bins)

    x_labels = []
    for start,stop in pairwise(bins):
        if stop==bins[-1]:
            x_labels.append(f'{start} a {stop}')
        elif stop==start+1:
            x_labels.append(f'{start}')
        else:
            x_labels.append(f'{start} a {stop-1}')
    
    bar_colors = ['tab:blue'] * len(heights)
    bar_colors[0] = 'tab:red'
    heights_argsort = heights.argsort()
    if heights_argsort[-1]==0:
        arg_max_count = heights_argsort[-2]
    else: 
        arg_max_count = heights_argsort[-1]
    bar_colors[arg_max_count] = 'tab:orange'
    
    fig, ax = plt.subplots()
    
    for i in range(len(heights)):
        p = ax.bar(x_labels[i],heights[i], color=bar_colors[i], alpha=0.85)
        ax.bar_label(p, label_type='center')
    
    # remove all the ticks (both axes), and tick labels on the Y axis
    ax.tick_params(bottom=False, left=False,
                   labelbottom=True, labelleft=False)
    
    # remove the frame of the chart
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    
    ax.set_ylabel('Quantidade de itens pesquisados')
    ax.set_xlabel('Resultados obtidos por item')
    ax.set_title('Quantidade de itens pesquisados e resultados obtidos')
    
    plt.show()


In [ ]:
def plot_top_sites(df,n_top=10):
    df_top_sites = df[df['netloc']!='-']
    df_top_sites = df_top_sites[['netloc','results']].groupby('netloc').sum().sort_values(by='results',ascending=False).head(n_top)
    top_sites = df_top_sites.index.values
    top_sites_count = df_top_sites.results.values
    
    top_sites_percentage = top_sites_count/top_sites_count.sum()
    top_sites_cum_percentage = top_sites_percentage.cumsum()
    bar_colors = ['tab:blue'] * n_top
    top_50_end = np.where(top_sites_cum_percentage>=0.5)[0][1]
    bottom_10 = np.where(top_sites_percentage<0.05)[0]
    for i in range(top_50_end):
        bar_colors[i] = 'tab:green'
    for i in bottom_10:
        bar_colors[i] = 'tab:red'
    
    y_pos = np.arange(len(top_sites))
    
    fig, ax = plt.subplots()
    
    for i in range(n_top):
        p = ax.barh(y_pos[i], top_sites_count[i], color = bar_colors[i], align='center', alpha=0.85)
        ax.bar_label(p,label_type='center')
        
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_yticks(y_pos, labels=top_sites)
    ax.set_title(f'Top {n_top} sites com mais resultados')
       
    # remove all the ticks (both axes), and tick labels on the Y axis
    ax.tick_params(bottom=False, left=False,
                   labelbottom=False, labelleft=True)
        
    # remove the frame of the chart
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
        
    plt.show()

In [ ]:
def plot_results_dashborard(df, bins=DEFAULT_BINS, n_top=DEFAULT_N_TOP_SITES):

    df['results'] = df['url'].apply(lambda x: 0 if x is None else 1)
    df['netloc'] = df['netloc'] = df_search_results_ean['url'].map(extract_site)

    # Figuras
    fig = plt.figure(figsize=(10, 8), layout='constrained')
    axs = fig.subplot_mosaic([['item_query_results','item_results_count'],
                              ['top_sites','top_sites']])
    for ax in axs:
        fig.get_layout_engine().set(hspace=0.1, wspace=0.1)
        
    ###########################    
    #   RESULTADOS POR ITEM   #
    ###########################
    results = df[['original_query','results']].groupby('original_query').sum().values.reshape(-1,)
    item_count_no_results = np.where(results==0)[0].shape[0]
    item_count_results = np.where(results>0)[0].shape[0]
    total_results = results.sum()
        
    labels = [f'Itens com resultado\n{item_count_results}', f'Itens sem resultado\n{item_count_no_results}']
    values = [item_count_results, item_count_no_results]
        
    startangle = 0
        
    colors = [(0.17254901960784313, 0.6274509803921569, 0.17254901960784313, 0.85),
              (0.8392156862745098, 0.15294117647058825, 0.1568627450980392, 0.85)]
    
    ax = axs['item_query_results']
    wedges, texts = axs['item_query_results'].pie(values, wedgeprops=dict(width=0.5), startangle=startangle, colors=colors)
        
    bbox_props = dict(boxstyle="round,pad=0.3", fc="w", ec="k", lw=0.72)
    kw = dict(arrowprops=dict(arrowstyle="-"),
              bbox=bbox_props, zorder=0, va="center")
        
    kw = dict(arrowprops=dict(arrowstyle="-"),bbox=bbox_props,
              zorder=0, va="center")
       
    for i, p in enumerate(wedges):
        ang = (p.theta2 - p.theta1)/2. + p.theta1
        y = np.sin(np.deg2rad(ang))
        x = np.cos(np.deg2rad(ang))
        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = "angle,angleA=0,angleB={}".format(ang)
        kw["arrowprops"].update({"connectionstyle": connectionstyle})
        # horizontalalignment = 'center'
        axs['item_query_results'].annotate(labels[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                                           horizontalalignment=horizontalalignment, **kw)
        
    axs['item_query_results'].annotate(f'{total_results:,d}\nresultados'.replace(',','.'), (0,0), horizontalalignment='center', verticalalignment='center')
    axs['item_query_results'].set_title('Quantidade de itens pesquisados e resultados obtidos')
    
    
    ############################    
    #   RESULTADOS POR FAIXA   #
    ############################
    search_results_count = df[['original_query','results']].groupby('original_query').sum().values.reshape(-1,)
    heights,_ = np.histogram(search_results_count,bins=bins)
    
    x_labels = []
    for start,stop in pairwise(bins):
        if stop==bins[-1]:
            x_labels.append(f'{start} a {stop}')
        elif stop==start+1:
            x_labels.append(f'{start}')
        else:
            x_labels.append(f'{start} a {stop-1}')
     
    bar_colors = ['tab:blue'] * len(heights)
    bar_colors[0] = 'tab:red'
    heights_argsort = heights.argsort()
    if heights_argsort[-1]==0:
        arg_max_count = heights_argsort[-2]
    else: 
        arg_max_count = heights_argsort[-1]
    bar_colors[arg_max_count] = 'tab:orange'

           
    for i in range(len(heights)):
        p = axs['item_results_count'].bar(x_labels[i],heights[i], color=bar_colors[i], alpha=0.85)
        axs['item_results_count'].bar_label(p, label_type='center')
        
    # remove all the ticks (both axes), and tick labels on the Y axis
    axs['item_results_count'].tick_params(bottom=False, left=False,
                                          labelbottom=True, labelleft=False)
        
    # remove the frame of the chart
    axs['item_results_count'].spines['left'].set_visible(False)
    axs['item_results_count'].spines['right'].set_visible(False)
    axs['item_results_count'].spines['top'].set_visible(False)
    axs['item_results_count'].spines['bottom'].set_visible(False)
    
    axs['item_results_count'].set_ylabel('Quantidade de itens pesquisados')
    axs['item_results_count'].set_xlabel('Resultados obtidos por item')
    axs['item_results_count'].set_title('Quantidade de itens pesquisados e resultados obtidos')
    
    ###########################    
    #   RESULTADOS POR SITE   #
    ###########################
    df_top_sites = df[df['netloc']!='-']
    df_top_sites = df_top_sites[['netloc','results']].groupby('netloc').sum().sort_values(by='results',ascending=False).head(n_top)
   
    top_sites = df_top_sites.index.values
    top_sites_count = df_top_sites.results.values
        
    top_sites_percentage = top_sites_count/top_sites_count.sum()
    top_sites_cum_percentage = top_sites_percentage.cumsum()
    bar_colors = ['tab:blue'] * n_top
    top_50_end = np.where(top_sites_cum_percentage>=0.5)[0][1]
    bottom_10 = np.where(top_sites_percentage<0.05)[0]
    for i in range(top_50_end):
        bar_colors[i] = 'tab:green'
    for i in bottom_10:
        bar_colors[i] = 'tab:red'
        
    y_pos = np.arange(len(top_sites))
        
    for i in range(n_top):
        p = axs['top_sites'].barh(y_pos[i], top_sites_count[i], color = bar_colors[i], align='center', alpha=0.85)
        axs['top_sites'].bar_label(p,label_type='center')
            
    axs['top_sites'].invert_yaxis()  # labels read top-to-bottom
    axs['top_sites'].set_yticks(y_pos, labels=top_sites)
    axs['top_sites'].set_title(f'Top {n_top} sites com mais resultados')
           
    # remove all the ticks (both axes), and tick labels on the Y axis
    axs['top_sites'].tick_params(bottom=False, left=False,
                                 labelbottom=False, labelleft=True)
            
    # remove the frame of the chart
    axs['top_sites'].spines['left'].set_visible(False)
    axs['top_sites'].spines['right'].set_visible(False)
    axs['top_sites'].spines['top'].set_visible(False)
    axs['top_sites'].spines['bottom'].set_visible(False)
    
    axs['item_query_results'].set_box_aspect(1)
    axs['item_results_count'].set_box_aspect(1)
    
    plt.show()

# Carga e preparação dos dados

## Importação dos arquivos

In [ ]:
# importar tabela completa do SCH (Dados Abertos)
file_sch = 'datasets/produtos_certificados.zip'
usecols = [0, 1, 2, 3, 11, 12, 13, 14, 15]
dtype = {'Número de Homologação': 'str',
         'CNPJ do Solicitante': 'str'}
df_sch = pd.read_csv(file_sch,sep=';',usecols=usecols,dtype=dtype)

# contar quantidade de números de homologação
df_sch['Quantidade de Número de Homologação']=df_sch[['Número de Homologação','Modelo']].groupby('Número de Homologação').transform('count')
df_sch = df_sch.sort_values(by=['Quantidade de Número de Homologação','Data da Homologação'],ascending=False).reset_index(drop=True)
df_sch.head()

In [ ]:
# importar subset do SCH
file_subset_sch = 'datasets/subset_sch.txt'
dtype = {'Número de Homologação': 'str'}
df_subset_sch = pd.read_csv(file_subset_sch,names=['Número de Homologação'],dtype=dtype)

# merge SCH subset and SCH to join columns "Modelo" and "Nome Comercial"
columns_to_merge = ['Número de Homologação', 'Modelo', 'Nome Comercial']
df_subset_sch=df_subset_sch.merge(df_sch[columns_to_merge])
df_subset_sch = df_subset_sch.fillna('#NULO#')
df_subset_sch = df_subset_sch.drop_duplicates()
df_subset_sch.head()

In [ ]:
# importa lista de EAN da Xiaomi
file_ean_xiaomi = 'datasets/ean_xiaomi.xlsx'
dtype = {'EAN': 'str'}
df_ean_xiaomi = pd.read_excel(file_ean_xiaomi,dtype=dtype)
df_ean_xiaomi.head()

In [ ]:
ean_to_search = df_ean_xiaomi['EAN']
ean_to_search.name='original_query'

ean_models_to_search = pd.Series(clean_models(df_ean_xiaomi['Descrição']))
ean_models_to_search.name='original_query'

sch_to_search = df_subset_sch['Número de Homologação'].drop_duplicates()
sch_to_search.name='original_query'

sch_models_to_search = pd.Series(list(set(df_subset_sch['Modelo'].unique().tolist()+df_subset_sch['Nome Comercial'].unique().tolist())))
sch_models_to_search.name='original_query'

In [ ]:
# carregar resultados de pesquisa
file_search_results = 'datasets/search_results/products_search_results.parquet'
df_search_results = pd.read_parquet(file_search_results)

# dividir resultados de pesquisa em grupos de pesquisa
file_search_results_ean = 'datasets/search_results/products_search_results_ean.parquet'
file_search_results_ean_models = 'datasets/search_results/products_search_results_ean_models.parquet'
file_search_results_sch = 'datasets/search_results/products_search_results_sch.parquet'
file_search_results_sch_models = 'datasets/search_results/products_search_results_sch_models.parquet'

# carregar os arquivos por grupo, se já tiverem sido separados anteriormente,
# caso contrário, separar e salvar o arquivo correspondente
if osp.exists(file_search_results_ean):
    df_search_results_ean = pd.read_parquet(file_search_results_ean)    
    print(f'File {file_search_results_ean} loaded')
else:
    df_search_results_ean = df_search_results.merge(ean_to_search)
    df_search_results_ean.to_parquet(file_search_results_ean)
    print(f'Dataframe df_search_results_sch created and saved to file {file_search_results_ean}')

if osp.exists(file_search_results_ean_models):
    df_search_results_ean_models = pd.read_parquet(file_search_results_ean_models)  
    print(f'File {file_search_results_ean_models} loaded')
else:
    df_search_results_ean_models = df_search_results.merge(ean_models_to_search)
    df_search_results_ean_models.to_parquet(file_search_results_ean_models)
    print(f'Dataframe df_search_results_sch created and saved to file {file_search_results_ean_models}')

if osp.exists(file_search_results_sch):
    df_search_results_sch = pd.read_parquet(file_search_results_sch)   
    print(f'File {file_search_results_sch} loaded')
else:
    df_search_results_sch = df_search_results.merge(sch_to_search)
    df_search_results_sch.to_parquet(file_search_results_sch)
    print(f'Dataframe df_search_results_sch created and saved to file {file_search_results_sch}')

if osp.exists(file_search_results_sch_models):
    df_search_results_sch_models = pd.read_parquet(file_search_results_sch_models)   
    print(f'File {file_search_results_sch_models} loaded')
else:
    df_search_results_sch_models = df_search_results.merge(sch_models_to_search)
    df_search_results_sch_models.to_parquet(file_search_results_sch_models)
    print(f'Dataframe df_search_results_sch created and saved to file {file_search_results_sch_models}')

# Análise

## Pesquisa por EAN

In [ ]:
plot_results_dashborard(df_search_results_ean)

## Pesquisa por código SCH

In [ ]:
plot_results_dashborard(df_search_results_sch)

## Pesquisa por modelo associado ao EAN

In [ ]:
plot_results_dashborard(df_search_results_ean_models)

## Pesquisa por modelo associado ao código SCH

In [ ]:
plot_results_dashborard(df_search_results_sch_models)